# Example 1 -Atlantic City example

In [1]:
# Written: sy Aug 2024
# License: BSD-2

"""
 Purpose: Testing Imputer
"""

import os
import sys
import copy
import json

import numpy as np

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
LOCATION_NAME = 'Atlantic City, NJ' #'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'AC.geojson' #'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 1

## Merging NSI and footprint

In [3]:
importer = Importer()

In [4]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [5]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data(region_boundary_object)


Searching for Atlantic City, NJ...
Found Atlantic City, Atlantic County, New Jersey, United States


INFO:root:
Getting National Structure Inventory (NSI) building data for the entered location...



Found a total of 6844 building points in NSI that are within the entered region of interest


In [6]:
scraper_class = importer.get_class('USA_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)


Searching for Atlantic City, NJ...
Found Atlantic City, Atlantic County, New Jersey, United States

Meshing the defined area...


Obtaining the number of elements in each cell: 100%|█████████████████████████████████████| 5/5 [00:00<00:00, 19.53it/s]



Meshing complete. Split Atlantic City into {len(final_cells)} cells


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]



Found a total of 4604 building footprints in Atlantic City


In [7]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True)


Getting National Structure Inventory (NSI) building data for the entered location...
Found a total of 3689 building points in NSI that match the footprint data.


##  Running imputation

In [8]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

C:\Users\SimCenter\Sangri\BrailsPlusPlus2\examples\inference\../..\brails\imputers\knn_imputer\knn_imputer.py:157: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bldg_properties_df = bldg_properties_df.replace("NA", np.nan, inplace=False)


Features with no reference data cannot be imputed. Removing them from the imputation target: buildingheight
Missing percentages among 4604 assets
fparea: 19.87%
numstories: 19.87%
erabuilt: 19.87%
repaircost: 19.87%
constype: 19.87%
occupancy: 19.87%
found_ht: 19.87%
splitlevel: 19.87%
basement: 38.18%
Primitive imputation done.
Running the main imputation. This may take a while.
Done imputation. It took 0.02 mins


## Temporarily Assigning features 

In the future, these features should be provided by the brails scraper

In [9]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({"FloodZone":'V'})

## Run inference

#### Rename features
so that the inferer module can understand

In [10]:
imputed_inventory.change_feature_names({'erabuilt': 'YearBuilt',
                                        'numstories': 'NumberOfStories',
                                        'basement': 'Basement',
                                        'occupancy': 'OccupancyClass',
                                        'splitlevel': 'SplitLevel',
                                        'found_ht': 'FirstFloorElevation' 
                                       })

#### Run inference

In [11]:
hurricaneInferer = importer.get_class("HazusInfererFlood")
inferer = hurricaneInferer(input_inventory=imputed_inventory ,clean_features=True,n_possible_worlds=NO_POSSIBLE_WORLDS)
hazus_inferred_inventory = inferer.infer()

Done inference. It took 0.03 mins


#### Check the outcomes

In [12]:
hazus_inferred_inventory.inventory[0].features

{'EffectiveFirstFloorElevation': 7.0,
 'OccupancyClass': 'RES1',
 'FloodType': 'cvz',
 'SplitLevel': 'No',
 'NumberOfStories': 2,
 'BasementType': 'bw'}

In [13]:
hazus_inferred_inventory.inventory[312].features

{'EffectiveFirstFloorElevation': 7.0,
 'OccupancyClass': 'COM1',
 'FloodType': 'cvz',
 'SplitLevel': 'No',
 'NumberOfStories': 1,
 'BasementType': 'bw'}